In [11]:
import pandas as pd
from nltk.tokenize import word_tokenize
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import pymorphy2
nltk.download('punkt')
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import numpy as np
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/operator3/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/operator3/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
df = pd.read_excel('profsa/Реестр 327 тыс. деклараций ЕП РФ без 140000-200000.xlsx', sheet_name='все ДС с кодами')

In [4]:
all_sents = []
for index, row in df.iterrows():
    all_sents.append(row['Общее наименование продукции'])

In [6]:
def make_sent_good(sent):
    
    #print(sent)
    tokens = word_tokenize(sent, language="russian")
    #print(tokens)
    filtered_tokens = []
    stop_words = stopwords.words("russian")
    for token in tokens:
        if token not in stop_words and token.isalpha():
#            parsed = morph.parse(token)
#            filtered_tokens.append(parsed[0].normal_form)
            filtered_tokens.append(token.lower())
    return filtered_tokens

In [7]:
dataset = [make_sent_good(sent) for sent in all_sents ]

In [10]:
k=0
classes = []
for cat in df['Раздел ЕП РФ (Код из ФГИС ФСА для подкатегории продукции)'].tolist():
    if ';' in cat:
        c =cat.split(';')[0]
        classes.append(c)
    else:
        classes.append(cat)

In [12]:
x_train, x_test, y_train, y_test = train_test_split(dataset, classes, test_size=0.1, random_state=42)

In [13]:
with open("train.txt", "w") as f:
    for x, y in zip(x_train, y_train):
        f.writelines("__label__"+y+" "+" ".join(x)+'\n')
with open("test.txt", "w") as f:
    for x, y in zip(x_test, y_test):
        f.writelines("__label__"+y+" "+" ".join(x)+"\n")

In [8]:
import fasttext

In [31]:
model = fasttext.train_supervised(input="train.txt", lr=0.3, epoch=100, wordNgrams=2, bucket=200000, dim=100, loss='ova')

In [32]:
k=0
for x, y in zip(x_test, y_test):
    pred = model.predict(' '.join(x))
    label = pred[0][0][9:]
    if label == y:
        k+=1
print(k / len(x_test))

0.9078038029063469


In [33]:
model.save_model("fast_text.pt")